In [7]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import linemerge
import numpy as np
import pandas as pd
import rasterio
from rasterio.features import rasterize
import os

In [12]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon

# File paths
stream_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-virtavesi_19-01-23_virtavesikapea.shp'
river_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-virtavesi_19-01-23_virtavesialue.shp'
out_dir = r'/Users/jpnousu/WBT_data/shapes/'
merged_file = os.path.join(out_dir, 'clipped_MTK-virtavesi_19-01-23_virtavesikapea_virtavesialue.shp')

# Load shapefiles
streams = gpd.read_file(stream_file)
rivers = gpd.read_file(river_file)

# Make sure CRS matches
if streams.crs != rivers.crs:
    rivers = rivers.to_crs(streams.crs)

# Keep only LineStrings from streams
streams_lines = streams[streams.geometry.type == 'LineString'].copy()

# Convert river polygons to their boundaries
river_boundaries = []
for geom in rivers.geometry:
    if geom is not None:
        if geom.type in ['Polygon', 'MultiPolygon']:
            river_boundary = geom.boundary  # LineString or MultiLineString
            river_boundaries.append(river_boundary)
        elif geom.type == 'LineString':
            river_boundaries.append(geom)

rivers_lines = gpd.GeoDataFrame(geometry=river_boundaries, crs=rivers.crs)

# Assign 'id' = 1
streams_lines['id'] = 1
rivers_lines['id'] = 1

# Keep only 'id' and 'geometry'
streams_lines = streams_lines[['id', 'geometry']]
rivers_lines = rivers_lines[['id', 'geometry']]

# Merge
merged = gpd.GeoDataFrame(pd.concat([streams_lines, rivers_lines], ignore_index=True), crs=streams.crs)

# Save merged shapefile
merged.to_file(merged_file)

print(f"Merged LineString shapefile (including river boundaries) saved to: {merged_file}")


Merged LineString shapefile (including river boundaries) saved to: /Users/jpnousu/WBT_data/shapes/clipped_MTK-virtavesi_19-01-23_virtavesikapea_virtavesialue.shp


/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_67863/1354867978.py:27: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ['Polygon', 'MultiPolygon']:


In [6]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import rasterio
from rasterio.features import rasterize
import os

# File paths
stream_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-virtavesi_19-01-23_virtavesikapea.shp'
river_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-virtavesi_19-01-23_virtavesialue.shp'
lake_file = r'/Users/jpnousu/WBT_data/shapes/clipped_MTK-vakavesi_24-08-01_jarvi.shp'
ref_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/keskipituus_vmi1x_1721.asc'
out_dir = r'/Users/jpnousu/WBT_data/rasters/'

# Output files
channels_file = os.path.join(out_dir, 'channels.asc')
lakes_file = os.path.join(out_dir, 'lakes.asc')

# Load geometries
streams = gpd.read_file(stream_file)
rivers = gpd.read_file(river_file)
lakes = gpd.read_file(lake_file)

# Open reference raster
with rasterio.open(ref_file) as ref:
    meta = ref.meta.copy()
    out_shape = (ref.height, ref.width)
    transform = ref.transform
    crs = ref.crs

# Reproject to match reference raster
for gdf in [streams, rivers, lakes]:
    if gdf.crs != crs:
        gdf.to_crs(crs, inplace=True)

# -------------------
# Rasterize channels (streams + rivers)
# -------------------
channel_shapes = []

# Streams (LineString)
channel_shapes += [(geom, 1) for geom in streams.geometry if geom is not None]

# Rivers (Polygon/MultiPolygon)
for geom in rivers.geometry:
    if geom is not None and isinstance(geom, (Polygon, MultiPolygon)):
        channel_shapes.append((geom, 1))

# Rasterize channels
channels_raster = rasterize(
    channel_shapes,
    out_shape=out_shape,
    transform=transform,
    fill=0,           # background = 0
    dtype='uint8',
    all_touched=True
)

# Update metadata
channel_meta = meta.copy()
channel_meta.update({
    "driver": "AAIGrid",  # save as ASCII
    "dtype": 'uint8',
    "count": 1,
    "nodata": 0
})

# Write channels
with rasterio.open(channels_file, 'w', **channel_meta) as dst:
    dst.write(channels_raster, 1)

print(f"Channels raster saved to: {channels_file}")

# -------------------
# Rasterize lakes
# -------------------
lake_shapes = []

# Lakes (Polygon/MultiPolygon)
for geom in lakes.geometry:
    if geom is not None and isinstance(geom, (Polygon, MultiPolygon)):
        lake_shapes.append((geom, 1))

# Rasterize lakes
lakes_raster = rasterize(
    lake_shapes,
    out_shape=out_shape,
    transform=transform,
    fill=0,
    dtype='uint8',
    all_touched=True
)

# Update metadata for lakes
lake_meta = meta.copy()
lake_meta.update({
    "driver": "AAIGrid",
    "dtype": 'uint8',
    "count": 1,
    "nodata": 0
})

# Write lakes
with rasterio.open(lakes_file, 'w', **lake_meta) as dst:
    dst.write(lakes_raster, 1)

print(f"Lakes raster saved to: {lakes_file}")


Channels raster saved to: /Users/jpnousu/WBT_data/rasters/channels.asc
Lakes raster saved to: /Users/jpnousu/WBT_data/rasters/lakes.asc
